In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

In [2]:
stardf = pd.read_csv("..//TG_stars.csv")
stardf.head (7)

,Vmag,Plx,e_Plx,B-V,Amag,TargetClass
0,9.10,3.54,1.39,0.482,16.845016,3
1,9.27,21.90,3.10,0.999,20.972221,5
2,6.61,2.81,0.63,-0.019,13.853532,1
3,8.06,7.75,0.97,0.370,17.506509,3
4,8.55,2.87,1.11,0.902,15.839409,4
5,12.31,18.80,4.99,1.336,23.680789,6
6,9.64,17.74,1.30,0.740,20.884768,4


In [3]:
stars_df_features = stardf.copy()

stars_df_features['Amag_SQ'] = stars_df_features['Amag']**2
stars_df_features['Vmag_SQ'] = stars_df_features['Vmag']**2
stars_df_features['B-V_SQ'] = stars_df_features['B-V']**2
stars_df_features['Plx_SQ'] = stars_df_features['Plx']**2
#stars_df_features['Sum_AV'] = stars_df_features['Amag'] + stars_df_features['Vmag']
#stars_df_features['Sub_AV'] = stars_df_features['Amag'] + stars_df_features['Vmag']

stars_df_features.head()

,Vmag,Plx,e_Plx,B-V,Amag,TargetClass,Amag_SQ,Vmag_SQ,B-V_SQ,Plx_SQ
0,9.10,3.54,1.39,0.482,16.845016,3,283.754574,82.8100,0.232324,12.5316
1,9.27,21.90,3.10,0.999,20.972221,5,439.834036,85.9329,0.998001,479.6100
2,6.61,2.81,0.63,-0.019,13.853532,1,191.920338,43.6921,0.000361,7.8961
3,8.06,7.75,0.97,0.370,17.506509,3,306.477840,64.9636,0.136900,60.0625
4,8.55,2.87,1.11,0.902,15.839409,4,250.886893,73.1025,0.813604,8.2369


In [5]:
X = stars_df_features.drop('TargetClass', axis=1, inplace=False)
Y = stars_df_features['TargetClass']
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, random_state=21, stratify=Y)

In [13]:
from imblearn.combine import SMOTETomek
from collections import Counter

In [14]:
counter= Counter(Y_train)
print('Before', counter)
smtom=SMOTETomek(random_state=21)
X_train_smtom, Y_train_smtom=smtom.fit_resample(X_train, Y_train)

counter1= Counter(Y_train_smtom)
print('After', counter1)

Before Counter({5: 2186, 3: 1966, 4: 1742, 2: 1028, 1: 447, 6: 346, 7: 18, 0: 8})
After Counter({0: 2172, 7: 2169, 6: 2076, 1: 2014, 2: 1924, 5: 1761, 3: 1756, 4: 1698})


In [16]:
Y_train_smtom.value_counts()

0    2172
7    2169
6    2076
1    2014
2    1924
5    1761
3    1756
4    1698
Name: TargetClass, dtype: int64

In [19]:
scaler = StandardScaler().fit(X_train_smtom)
X_train_scaled = scaler.transform(X_train_smtom)
X_test_scaled = scaler.transform(X_test)

clf =RandomForestClassifier(n_estimators=200, max_features="log2", max_depth=10).fit(X_train_scaled, Y_train_smtom)

y_pred=clf.predict(X_test_scaled)

print(f'RFC Training Data : {clf.score(X_train_scaled,Y_train_smtom)}')
print(f'RFC Testing Data: {clf.score(X_test_scaled,Y_test)}')

RFC Training Data : 0.9059730250481696
RFC Testing Data: 0.743801652892562


In [24]:
classifier = clf
classifier
classifier.fit(X_train_smtom, Y_train_smtom)

RandomForestClassifier(max_depth=10, max_features='log2', n_estimators=200)

In [25]:
predictions = classifier.predict(X_test)
print(f"First 10 predictions:   {predictions[:10]}")
print(f"First 10 actual labels: {Y_test[:10].tolist()}")

First 10 predictions:   [3 7 2 2 4 2 5 3 5 2]
First 10 actual labels: [3, 3, 2, 1, 5, 1, 5, 3, 5, 2]


2858    3
9441    3
5607    2
8879    1
2799    5
       ..
7918    5
6382    5
3360    2
35      3
5185    1
Name: TargetClass, Length: 1936, dtype: int64

In [26]:
y_true = Y_test
y_pred = classifier.predict(X_test)
confusion_matrix(y_true, y_pred)

array([[  0,   1,   1,   0,   0,   0,   0,   0],
       [  5,  71,  24,   4,   5,   0,   1,   2],
       [  2,  46, 190,  16,   3,   0,   0,   0],
       [  4,  12,  21, 401,  49,   2,   1,   1],
       [  3,   2,   1,  45, 300,  78,   2,   5],
       [  0,   0,   1,   2,  55, 409,  64,  16],
       [  0,   0,   0,   1,   1,  11,  68,   6],
       [  0,   0,   0,   0,   0,   0,   1,   3]], dtype=int64)

In [27]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.54      0.63      0.58       112
           2       0.80      0.74      0.77       257
           3       0.86      0.82      0.84       491
           4       0.73      0.69      0.71       436
           5       0.82      0.75      0.78       547
           6       0.50      0.78      0.61        87
           7       0.09      0.75      0.16         4

    accuracy                           0.74      1936
   macro avg       0.54      0.64      0.56      1936
weighted avg       0.77      0.74      0.75      1936

